In [7]:
import pandas as pd
import numpy as np
import scipy.sparse as sps

from scipy.sparse import *

In [42]:
import time

# BPR

In [8]:
urm_path = '../../content/data_train.csv'

urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]

In [9]:
urm_all_df.head(10)

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
5,1,187,1.0
6,1,205,1.0
7,1,222,1.0
8,1,237,1.0
9,1,354,1.0


In [10]:
print ("The number of interactions is {}".format(len(urm_all_df)))

The number of interactions is 478730


In [11]:
userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024

Average interactions per user 37.88
Average interactions per item 21.54

Sparsity 99.83 %


Not all users are in the urm:

In [12]:
mapped_id, original_id = pd.factorize(urm_all_df["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(urm_all_df["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

We now replace the IDs in the dataframe and we are ready to use the data

In [13]:
urm_all_df["UserID"] = urm_all_df["UserID"].map(user_original_ID_to_index)
urm_all_df["ItemID"] = urm_all_df["ItemID"].map(item_original_ID_to_index)

In [14]:
urm_all_df.head(n=10)

,UserID,ItemID,Interaction
0,0,0,1.0
1,0,1,1.0
2,0,2,1.0
3,0,3,1.0
4,0,4,1.0
5,0,5,1.0
6,0,6,1.0
7,0,7,1.0
8,0,8,1.0
9,0,9,1.0


In [15]:
urm_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

urm_all

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [16]:
urm_all.tocsr()

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [17]:
def precision(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [18]:
def evaluate_algorithm(URM_test, recommender_object, at=5):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

            recommended_items = recommender_object.recommend(user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP))

---

# Splitting Train and Validation set

In [19]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix

def train_test_split_urm(urm, train_percentage=0.8, seed=None):
    """
    Splits the User-Item Interaction Matrix (URM) into training and testing sets.

    Parameters:
    - urm (csr_matrix): The User-Item Interaction Matrix.
    - train_percentage (float): The percentage of interactions to include in the training set.
    - seed (int): Seed for reproducibility.

    Returns:
    - train_urm (csr_matrix): Training User-Item Interaction Matrix.
    - test_urm (csr_matrix): Testing User-Item Interaction Matrix.
    """

    if not (0 < train_percentage < 1):
        raise ValueError("train_percentage must be between 0 and 1 exclusive")

    # Set seed for reproducibility
    if seed is not None:
        np.random.seed(seed)

    # Get the non-zero indices (user and item indices)
    non_zero_indices = urm.nonzero()

    # Get the number of non-zero interactions
    num_interactions = len(non_zero_indices[0])

    # Randomly shuffle the indices
    shuffled_indices = np.arange(num_interactions)
    np.random.shuffle(shuffled_indices)

    # Calculate the number of interactions for the training set
    num_train_interactions = int(train_percentage * num_interactions)

    # Split the indices into training and testing sets
    train_indices = shuffled_indices[:num_train_interactions]
    test_indices = shuffled_indices[num_train_interactions:]

    # Create masks for indexing
    train_mask = np.zeros(num_interactions, dtype=bool)
    test_mask = np.zeros(num_interactions, dtype=bool)

    train_mask[train_indices] = True
    test_mask[test_indices] = True

    # Extract values and create new matrices
    train_data = urm.data[train_mask]
    train_rows = non_zero_indices[0][train_mask]
    train_cols = non_zero_indices[1][train_mask]

    test_data = urm.data[test_mask]
    test_rows = non_zero_indices[0][test_mask]
    test_cols = non_zero_indices[1][test_mask]

    # Create new matrices using the extracted values
    train_urm = coo_matrix((train_data, (train_rows, train_cols)), shape=urm.shape).tocsr()
    test_urm = coo_matrix((test_data, (test_rows, test_cols)), shape=urm.shape).tocsr()

    return train_urm, test_urm


Split the dataset into train and validation. We set the split to 0.8-0.2 as a standard. It's an hyperparameter: we should fine tune it.

In [20]:
# Assuming urm is your User-Item Interaction Matrix in csr_matrix format
train_urm, test_urm = train_test_split_urm(urm_all, train_percentage=0.8, seed=42)

-------------------------------------------------------------------------------

# BRP implementation #1

## Step 1: We create a dense similarity matrix, initialized as zero

In [23]:
n_users, n_items = train_urm.shape

In [25]:
item_item_S = np.zeros((n_items, n_items), dtype = np.float32)
item_item_S

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Step 2: We sample a triplet

#### Create a mask of positive interactions. How to build it depends on the data

In [27]:
URM_mask = train_urm.copy()
URM_mask.data[URM_mask.data < 1.0] = 0

URM_mask.eliminate_zeros()
URM_mask

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>

In [28]:
user_id = np.random.choice(n_users)
user_id

5762

### Get user seen items and choose one

In [29]:
user_seen_items = URM_mask.indices[URM_mask.indptr[user_id]:URM_mask.indptr[user_id+1]]
user_seen_items

array([   87,  1946,  2433,  3330,  5086,  6903, 11507, 15193, 15617,
       16759, 20931])

In [30]:
pos_item_id = np.random.choice(user_seen_items)
pos_item_id

15193

### To select a negative item it's faster to just try again then to build a mapping of the non-seen items

In [31]:
neg_item_selected = False

# It's faster to just try again then to build a mapping of the non-seen items
while (not neg_item_selected):
    neg_item_id = np.random.randint(0, n_items)

    if (neg_item_id not in user_seen_items):
        neg_item_selected = True

neg_item_id

4972

## Step 2 - Computing prediction

#### The prediction depends on the model: SLIM, Matrix Factorization... 
#### Note that here the data is implicit so we do not multiply for the user rating, because it is always 1, we just sum the similarities of the seen items.

In [32]:
x_ui = item_item_S[pos_item_id, user_seen_items].sum()
x_uj = item_item_S[neg_item_id, user_seen_items].sum()

print("x_ui is {:.4f}, x_uj is {:.4f}".format(x_ui, x_uj))

x_ui is 0.0000, x_uj is 0.0000


## Step 3 - Computing gradient

#### The gradient depends on the objective function: RMSE, BPR... 

In [33]:
x_uij = x_ui - x_uj
x_uij

0.0

#### The original BPR paper uses the logarithm of the sigmoid of x_ij, whose derivative is the following

In [34]:
sigmoid_item = 1 / (1 + np.exp(x_uij))
sigmoid_item

0.5

## Step 4 - Update model

#### How to update depends on the model itself, here we have just one paramether, the similarity matrix, so we perform just one update. In matrix factorization we have two.

#### We need a learning rate, which influences how fast the model will change. Small ones lead to slower convergence but often higher results

In [35]:
learning_rate = 1e-3

item_item_S[pos_item_id, user_seen_items] += learning_rate * sigmoid_item
item_item_S[pos_item_id, pos_item_id] = 0

item_item_S[neg_item_id, user_seen_items] -= learning_rate * sigmoid_item
item_item_S[neg_item_id, neg_item_id] = 0

#### Usually there is no relevant change in the scores over a single iteration

In [37]:
x_i = item_item_S[pos_item_id, user_seen_items].sum()
x_j = item_item_S[neg_item_id, user_seen_items].sum()

print("x_i is {:.4f}, x_j is {:.4f}".format(x_i, x_j))

x_i is 0.0050, x_j is -0.0055


## Now we put everything in a training loop

In [38]:
def sample_triplet():
    
    non_empty_user = False
    
    while not non_empty_user:
        user_id = np.random.choice(n_users)
        user_seen_items = URM_mask.indices[URM_mask.indptr[user_id]:URM_mask.indptr[user_id+1]]
        
        if len(user_seen_items)>0:
            non_empty_user = True

    pos_item_id = np.random.choice(user_seen_items)

    neg_item_selected = False

    # It's faster to just try again then to build a mapping of the non-seen items
    while (not neg_item_selected):
        neg_item_id = np.random.randint(0, n_items)

        if (neg_item_id not in user_seen_items):
            neg_item_selected = True

    return user_id, pos_item_id, neg_item_id    

In [39]:
def train_one_epoch(item_item_S, learning_rate):

    start_time = time.time()
    for sample_num in range(n_users):

        # Sample triplet
        user_id, pos_item_id, neg_item_id = sample_triplet()
        
        user_seen_items = URM_mask.indices[URM_mask.indptr[user_id]:URM_mask.indptr[user_id+1]]

        # Prediction
        x_ui = item_item_S[pos_item_id, user_seen_items].sum()
        x_uj = item_item_S[neg_item_id, user_seen_items].sum()
        
        # Gradient
        x_uij = x_ui - x_uj

        sigmoid_item = 1 / (1 + np.exp(x_uij))
        
        # Update
        item_item_S[pos_item_id, user_seen_items] += learning_rate * sigmoid_item
        item_item_S[pos_item_id, pos_item_id] = 0

        item_item_S[neg_item_id, user_seen_items] -= learning_rate * sigmoid_item
        item_item_S[neg_item_id, neg_item_id] = 0

        # Print some stats
        if (sample_num +1)% 50000 == 0 or (sample_num +1) == n_users:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds. Samples per second {:.2f}".format(sample_num+1, elapsed_time, samples_per_second))
         
            
    return item_item_S, samples_per_second

In [43]:
learning_rate = 1e-6
    
item_item_S = np.zeros((n_items, n_items), dtype = np.float32)

for n_epoch in range(5):
    item_item_S, samples_per_second = train_one_epoch(item_item_S, learning_rate)

Iteration 12638 in 2.81 seconds. Samples per second 4505.19
Iteration 12638 in 2.99 seconds. Samples per second 4226.43
Iteration 12638 in 2.86 seconds. Samples per second 4415.45
Iteration 12638 in 2.81 seconds. Samples per second 4493.56
Iteration 12638 in 2.35 seconds. Samples per second 5372.89


In [44]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 14889.55 seconds, or 248.16 minutes


---

# BPR for MF

### What do we need for BPRMF?

* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience

## Step 1: We create the dense latent factor matrices

In [46]:
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [47]:
user_factors

array([[0.33265496, 0.46135691, 0.13712645, ..., 0.87134154, 0.51632778,
        0.16276541],
       [0.35773688, 0.53660301, 0.63453869, ..., 0.57485346, 0.30774007,
        0.47227938],
       [0.74147919, 0.42810152, 0.28685689, ..., 0.74410532, 0.87663233,
        0.11340114],
       ...,
       [0.89671821, 0.0914848 , 0.98283967, ..., 0.39069518, 0.73609471,
        0.98083937],
       [0.26721836, 0.0554383 , 0.93551587, ..., 0.86695942, 0.04400127,
        0.81796045],
       [0.63857667, 0.93340402, 0.40422246, ..., 0.68182548, 0.86021713,
        0.62757575]])

In [48]:
item_factors

array([[0.36511249, 0.59567211, 0.47673196, ..., 0.71778639, 0.98414289,
        0.39063499],
       [0.29574783, 0.30495207, 0.51009617, ..., 0.13274052, 0.34374374,
        0.01170671],
       [0.92824869, 0.8915113 , 0.71124745, ..., 0.72666796, 0.61929917,
        0.43411906],
       ...,
       [0.5940921 , 0.21940876, 0.99822777, ..., 0.93599639, 0.58309985,
        0.11041542],
       [0.5447615 , 0.55836027, 0.01468073, ..., 0.36864283, 0.5133718 ,
        0.93665372],
       [0.17466332, 0.10995398, 0.5706645 , ..., 0.96265034, 0.50723536,
        0.50152942]])

## Step 2 - Computing prediction

In [49]:
user_id, pos_item_id, neg_item_id = sample_triplet()
(user_id, pos_item_id, neg_item_id)

(11678, 5521, 14869)

In [50]:
x_ui = np.dot(user_factors[user_id,:], item_factors[pos_item_id,:])
x_uj = np.dot(user_factors[user_id,:], item_factors[neg_item_id,:])

print("x_ui is {:.4f}, x_uj is {:.4f}".format(x_ui, x_uj))

x_ui is 2.5084, x_uj is 2.5521


## Step 3 - Computing gradient

In [51]:
x_uij = x_ui - x_uj
x_uij

-0.043676931128342034

In [52]:
sigmoid_item = 1 / (1 + np.exp(x_uij))
sigmoid_item

0.5109174972515096

## Step 4 - Update model

In [53]:
regularization = 1e-4
learning_rate = 1e-2

H_i = item_factors[pos_item_id,:]
H_j = item_factors[neg_item_id,:]
W_u = user_factors[user_id,:]


user_factors[user_id,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
item_factors[pos_item_id,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
item_factors[neg_item_id,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)

In [54]:
x_ui = np.dot(user_factors[user_id,:], item_factors[pos_item_id,:])
x_uj = np.dot(user_factors[user_id,:], item_factors[neg_item_id,:])

print("x_i is {:.4f}, x_j is {:.4f}".format(x_ui, x_uj))

x_i is 2.5345, x_j is 2.5260


In [55]:
x_uij = x_ui - x_uj
x_uij

0.008518594277170166

In [56]:
def train_one_epoch(user_factors, item_factors, learning_rate):

    start_time = time.time()
    for sample_num in range(n_users):

        # Sample triplet
        user_id, pos_item_id, neg_item_id = sample_triplet()
        
        # Prediction
        x_ui = np.dot(user_factors[user_id,:], item_factors[pos_item_id,:])
        x_uj = np.dot(user_factors[user_id,:], item_factors[neg_item_id,:])
        
        # Gradient
        x_uij = x_ui - x_uj

        sigmoid_item = 1 / (1 + np.exp(x_uij))
                
        H_i = item_factors[pos_item_id,:]
        H_j = item_factors[neg_item_id,:]
        W_u = user_factors[user_id,:]


        user_factors[user_id,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
        item_factors[pos_item_id,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
        item_factors[neg_item_id,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)

        # Print some stats
        if (sample_num +1)% 50000 == 0 or (sample_num +1) == n_users:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds. Samples per second {:.2f}".format(sample_num+1, elapsed_time, samples_per_second))
        
    return user_factors, item_factors, samples_per_second

In [57]:
learning_rate = 1e-6
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

for n_epoch in range(5):
    user_factors, item_factors, samples_per_second = train_one_epoch(user_factors, item_factors, learning_rate)

Iteration 12638 in 2.17 seconds. Samples per second 5823.50
Iteration 12638 in 2.50 seconds. Samples per second 5058.89
Iteration 12638 in 2.45 seconds. Samples per second 5160.08
Iteration 12638 in 2.20 seconds. Samples per second 5754.53
Iteration 12638 in 2.04 seconds. Samples per second 6190.88


In [60]:
x_ui

2.5344849765745803

---

# Predictions

In [64]:
# After training, you can use the learned user and item factors for making predictions
# For example, to predict the score for a specific user and item:
user_id = 0
item_id = 1
prediction = np.dot(user_factors[user_id, :], item_factors[item_id, :])
print(f"Prediction for user {user_id} and item {item_id}: {prediction}")

Prediction for user 0 and item 1: 3.132971946898877
